In [20]:
import requests
import pytz
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime
from pandas_datareader import data as dat

import fix_yahoo_finance as yf
yf.pdr_override()


SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
START = pd.to_datetime('2010-01-01').tz_localize('US/Eastern')
END = pd.to_datetime('2018-11-11').tz_localize('US/Eastern')


def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    
    req = requests.get(SITE, headers=hdr)
    soup = BeautifulSoup(req.content, "html.parser")

    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0:
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers


def download_ohlc(sector_tickers, start, end):
    sector_ohlc = {}
    for sector, tickers in sector_tickers.items():
        print('Downloading data from Yahoo for %s sector' % sector)
        print(tickers)
        data = dat.DataReader(tickers, 'yahoo', start, end)
        for item in ['Open', 'High', 'Low']:
            try:
                
                data[item] = data[item] * data['Adj Close'] / data['Close']
#         data.rename(items={'Open': 'open', 'High': 'high', 'Low': 'low',
#                            'Adj Close': 'close', 'Volume': 'volume'},
#                     inplace=True)
#         data.drop(['Close'], inplace=True)
            except KeyError:
                pass
        sector_ohlc[sector] = data
    print('Finished downloading data')
    return sector_ohlc


def store_HDF5(sector_ohlc, path):
    with pd.get_store(path) as store:
        for sector, ohlc in sector_ohlc.items():
            store[sector] = ohlc


def get_snp500():
    sector_tickers = scrape_list(SITE)
    sector_ohlc = download_ohlc(sector_tickers, START, END)
    store_HDF5(sector_ohlc, 'snp500.h5')


if __name__ == '__main__':
    get_snp500()

['MMM', 'ALK', 'ALLE', 'AAL', 'AME', 'AOS', 'ARNC', 'BA', 'CHRW', 'CAT', 'CTAS', 'CPRT', 'CSX', 'CMI', 'DE', 'DAL', 'DOV', 'ETN', 'EMR', 'EFX', 'EXPD', 'FAST', 'FDX', 'FLS', 'FLR', 'FTV', 'FBHS', 'GD', 'GE', 'GWW', 'HRS', 'HON', 'HII', 'INFO', 'ITW', 'IR', 'JEC', 'JBHT', 'JCI', 'KSU', 'LLL', 'LMT', 'MAS', 'NLSN', 'NSC', 'NOC', 'PCAR', 'PH', 'PNR', 'PWR', 'RTN', 'RSG', 'RHI', 'ROK', 'COL', 'ROL', 'ROP', 'SNA', 'LUV', 'SWK', 'SRCL', 'TXT', 'TDG', 'UNP', 'UAL', 'UPS', 'URI', 'UTX', 'VRSK', 'WM', 'XYL']
['ABT', 'ABBV', 'ABMD', 'AET', 'A', 'ALXN', 'ALGN', 'AGN', 'ABC', 'AMGN', 'ANTM', 'BAX', 'BDX', 'BIIB', 'BSX', 'BMY', 'CAH', 'CELG', 'CNC', 'CERN', 'CI', 'COO', 'CVS', 'DHR', 'DVA', 'XRAY', 'EW', 'ESRX', 'GILD', 'HCA', 'HSIC', 'HOLX', 'HUM', 'IDXX', 'ILMN', 'INCY', 'ISRG', 'IQV', 'JNJ', 'LH', 'LLY', 'MCK', 'MDT', 'MRK', 'MTD', 'MYL', 'NKTR', 'PKI', 'PRGO', 'PFE', 'DGX', 'REGN', 'RMD', 'SYK', 'TMO', 'UNH', 'UHS', 'VAR', 'VRTX', 'WAT', 'WCG', 'ZBH', 'ZTS']
['ACN', 'ADBE', 'AMD', 'AKAM', 'ADS'

KeyError: 'Date'

In [26]:
import requests
import pytz
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime
from pandas_datareader import data as pdr

import fix_yahoo_finance as yf
yf.pdr_override()


SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
START = pd.to_datetime('2009-01-01').tz_localize('US/Eastern')
END = pd.to_datetime('2018-11-11').tz_localize('US/Eastern')


def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    
    req = requests.get(SITE, headers=hdr)
    soup = BeautifulSoup(req.content, "html.parser")

    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0:
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers


def download_ohlc(sector_tickers, start, end):
    sector_ohlc = []
    for sector, tickers in sector_tickers.items():
        print('Downloading data from Yahoo for %s sector' % sector)
        
        data = pdr.get_data_yahoo(tickers, start=START, end=END)
        sector_ohlc.append(data)
    print('Finished downloading data')
    print(sector_ohlc)
    return sector_ohlc


def store_HDF5(sector_ohlc):
    sector_ohlc.to_csv('full_data_sp500.csv', index = False)


def get_snp500():
    sector_tickers = scrape_list(SITE)
    sector_ohlc = download_ohlc(sector_tickers, START, END)
    store_HDF5(sector_ohlc)


# if __name__ == '__main__':
#     get_snp500()

In [28]:
sector = scrape_list(SITE)
sector_ohlc = download_ohlc(sector, START, END)

[*********************100%***********************]  71 of 71 downloaded
[*********************100%***********************]  63 of 63 downloaded
[*********************100%***********************]  67 of 67 downloaded
[*********************100%***********************]  26 of 26 downloaded
[*********************100%***********************]  65 of 65 downloaded
[*********************100%***********************]  29 of 29 downloaded
[*********************100%***********************]  67 of 67 downloaded
[*********************100%***********************]  24 of 24 downloaded
[*********************100%***********************]  32 of 32 downloaded
[*********************100%***********************]  32 of 32 downloaded
[*********************100%***********************]  29 of 29 downloaded
Finished downloading data
[                 Open                                                         \
                  AAL        ALK       ALLE        AME        AOS       ARNC   
Date                 

In [57]:
len(sector_ohlc)

11

In [58]:
a = sector_ohlc[0].unstack(level = 0).unstack(level = 0).reset_index()
for j in range(1, len(sector_ohlc)):
    b = sector_ohlc[j].unstack(level = 0).unstack(level = 0).reset_index()
    a = a.append(b, ignore_index = True)

In [67]:
a = a.rename(index = str, columns = {'level_0': 'ticker'})

In [68]:
a.to_csv('sp500_full.csv', index = False)

In [69]:
a.columns

Index(['ticker', 'Date', 'Adj Close', 'Close', 'High', 'Low', 'Open',
       'Volume'],
      dtype='object')

In [66]:
a

,level_0,Date,Adj Close,Close,High,Low,Open,Volume
0,AAL,2008-12-31,7.405228,7.730000,7.870000,7.480000,7.480000,4194100.0
1,AAL,2009-01-02,8.037499,8.390000,8.480000,7.670000,7.730000,5167000.0
2,AAL,2009-01-05,7.980019,8.330000,8.390000,7.960000,8.380000,3457100.0
3,AAL,2009-01-06,8.679349,9.060000,9.210000,8.130000,8.150000,5731000.0
4,AAL,2009-01-07,8.698509,9.080000,9.470000,8.660000,8.660000,5468900.0
5,AAL,2009-01-08,9.167923,9.570000,9.670000,8.620000,8.950000,5196400.0
6,AAL,2009-01-09,8.621870,9.000000,9.380000,8.880000,9.340000,5113200.0
7,AAL,2009-01-12,8.181195,8.540000,9.160000,8.270000,9.150000,3306600.0
8,AAL,2009-01-13,7.922542,8.270000,8.660000,8.110000,8.350000,3743800.0
9,AAL,2009-01-14,7.079513,7.390000,8.160000,7.250000,8.120000,6360700.0
